# Import and setup

In [9]:
import numpy as np
import matplotlib.pyplot as plt

import few

from few.trajectory.inspiral import EMRIInspiral
from few.trajectory.ode import SchwarzEccFlux, KerrEccEqFlux
# from few.amplitude.romannet import RomanAmplitude
from few.amplitude.ampinterp2d import AmpInterpKerrEccEq
from few.summation.interpolatedmodesum import InterpolatedModeSum, CubicSplineInterpolant


from few.utils.ylm import GetYlms
from few.utils.modeselector import ModeSelector
from few.summation.interpolatedmodesum import CubicSplineInterpolant
from few import get_file_manager

from few.waveform import (
    FastKerrEccentricEquatorialFlux,
    FastSchwarzschildEccentricFlux, 
    SlowSchwarzschildEccentricFlux, 
    Pn5AAKWaveform,
    GenerateEMRIWaveform
)

from few.utils.geodesic import get_fundamental_frequencies

import GWfuncs
import gc
import pickle
import os
import cupy as cp
import multiprocessing as mp
from multiprocessing import Queue, Process
from functools import partial
from SNR_tutorial_utils import LISA_Noise
from lisatools.sensitivity import *

from few.utils.constants import YRSID_SI, Gpc, MRSUN_SI



# import pandas as pd
# tune few configuration
cfg_set = few.get_config_setter(reset=True)
cfg_set.set_log_level("info");

In [2]:
# Parameters
m1 = 1e6 #M
m2 = 1e1 #mu
a = 0.5
p0 = 9.5
e0 = 0.2
theta = np.pi / 3.0 
phi = np.pi / 4.0  
dt = 10.0
T = 1
xI0 = 1.0 
#in the paper xI0 = 0.866, but that would be non-equatorial case

use_gpu = True 
traj = EMRIInspiral(func=KerrEccEqFlux, force_backend="cuda12x", use_gpu=use_gpu) #theres npoints flag here
amp = AmpInterpKerrEccEq(force_backend="cuda12x") # default lmax=10, nmax=55
interpolate_mode_sum = InterpolatedModeSum(force_backend="cuda12x")
ylm_gen = GetYlms(include_minus_m=False, force_backend="cuda12x")

# Generate amplitudes, trajectory, etc.

In [28]:
N_traj = 100 # change amount of points here 
delta_T = T*YRSID_SI/N_traj 

In [29]:
# Calc trajectory
(t, p, e, x, Phi_phi, Phi_theta, Phi_r) = traj(m1, m2, a, p0, e0, xI0, T=T, dt=delta_T, upsample=True) 

t_gpu = cp.asarray(t)

# Get amplitudes along trajectory
teuk_modes = amp(a, p, e, x)

# Get Ylms
ylms = ylm_gen(amp.unique_l, amp.unique_m, theta, phi).copy()[amp.inverse_lm]

In [30]:
dist = 1.0 # Gpc
factor = dist * Gpc / (m2 * MRSUN_SI)
N_traj = teuk_modes.shape[0]  # number of trajectory points

# Interpolate -> sum (one full waveform)

Still have problems with NaNs

In [31]:
%%time

t_gpu = cp.asarray(t)

# need to prepare arrays for sum with all modes due to +/- m setup
ls = amp.l_arr[: teuk_modes.shape[1]]
ms = amp.m_arr[: teuk_modes.shape[1]]
ns = amp.n_arr[: teuk_modes.shape[1]]

keep_modes = np.arange(teuk_modes.shape[1])
temp2 = keep_modes * (keep_modes < amp.num_m0) + (keep_modes + amp.num_m_1_up) * (
    keep_modes >= amp.num_m0
) 

ylmkeep = np.concatenate([keep_modes, temp2])
ylms_in = ylms[ylmkeep]
teuk_modes_in = teuk_modes

cp.cuda.Stream.null.synchronize()

CPU times: user 3.78 ms, sys: 17 μs, total: 3.8 ms
Wall time: 3.14 ms


In [32]:
%%time

# perform summation
waveform1 = interpolate_mode_sum(
    t_gpu,
    teuk_modes_in,
    ylms_in,
    traj.integrator_spline_t,
    traj.integrator_spline_phase_coeff[:, [0, 2]],
    ls,
    ms,
    ns,
    dt=dt,
    T=T,
)

cp.cuda.Stream.null.synchronize()

CPU times: user 13.6 s, sys: 44.9 ms, total: 13.6 s
Wall time: 13.6 s


In [33]:
waveform1[~np.isnan(waveform1)]

array([0.21930317+0.1927499j , 0.21930321+0.19274987j,
       0.21930332+0.19274977j, ..., 0.00028628+0.16310596j,
       0.0002863 +0.16310597j, 0.00028632+0.16310597j], shape=(347140,))

# Interpolate 1 mode with interpmodesum

In [34]:
%%time

# interpolate only one mode 
waveform1_1 = interpolate_mode_sum(
    t_gpu,
    teuk_modes_in[:, 2:3],
    ylms_in[1:2],
    traj.integrator_spline_t,
    traj.integrator_spline_phase_coeff[:, [0, 2]],
    ls[1:2],
    ms[1:2],
    ns[1:2],
    dt=dt,
    T=T,
)

cp.cuda.Stream.null.synchronize()

CPU times: user 7 ms, sys: 1.98 ms, total: 8.97 ms
Wall time: 8.76 ms


In [35]:
waveform1_1[~np.isnan(waveform1_1)]

array([-1.16315501e-19-4.08935528e-20j,  7.05224931e-20-1.01134188e-19j,
        5.76304600e-20+1.08996532e-19j, ...,
       -5.40243339e-20+5.24450841e-20j, -4.90025460e-20-5.71650714e-20j,
        6.00743703e-20-4.53888127e-20j], shape=(347140,))

In [38]:
np.sum(~np.isnan(waveform1_1))/len(waveform1_1) * 100

array(11.22450002)

# Interpolate 1 mode with CubicSpineInterpolant